In [59]:
import pandas as pd

In [60]:
Porcentaje_empleo_desempleo_participacion= pd.read_csv(r'Etl\Employment, unemployment, and participation rates by place of birth and sex.csv')

In [61]:
Porcentaje_empleo_desempleo_participacion.drop(['BIRTH', 'GENDER', 'RATE', 'YEAR', 'Flag Codes', 'Flags'], axis=1, inplace= True)

In [62]:
Porcentaje_empleo_desempleo_participacion.drop(columns=Porcentaje_empleo_desempleo_participacion.columns[6:12],axis=1,inplace= True)

In [63]:
Porcentaje_empleo_desempleo_participacion.rename(columns={'COUNTRY':'codigo_pais','Country':'pais',
                          'Place of birth':'lugar_de_nacimiento', 'Gender':'genero',
                          'Rate': 'referencia_tasas', 'Year': 'anio', 'Value': 'valor'}, inplace=True)

In [64]:
Porcentaje_empleo_desempleo_participacion= Porcentaje_empleo_desempleo_participacion[(Porcentaje_empleo_desempleo_participacion['genero']== 'Total')]

In [65]:
Porcentaje_empleo_desempleo_participacion.drop(['genero'], axis=1, inplace= True)

In [66]:
Porcentaje_empleo_desempleo_participacion_empleados = Porcentaje_empleo_desempleo_participacion[Porcentaje_empleo_desempleo_participacion['referencia_tasas'] == 'Employment rate'].rename(columns={'valor':"empleados"}).drop(['referencia_tasas'], axis=1)

In [67]:
Porcentaje_empleo_desempleo_participacion_desempleados = Porcentaje_empleo_desempleo_participacion[Porcentaje_empleo_desempleo_participacion['referencia_tasas'] == 'Unemployment rate'].rename(columns={'valor':"desempleados"}).drop(['referencia_tasas'], axis=1)

In [68]:
Porcentaje_empleo_desempleo_participacion_participacion = Porcentaje_empleo_desempleo_participacion[Porcentaje_empleo_desempleo_participacion['referencia_tasas'] == 'Participation rate'].rename(columns={'valor':"participacion"}).drop(['referencia_tasas'], axis=1)

In [ ]:
indices = Porcentaje_empleo_desempleo_participacion[(Porcentaje_empleo_desempleo_participacion["codigo_pais"]=="OECD")].index
Porcentaje_empleo_desempleo_participacion.drop(indices,inplace=True)

In [ ]:
indices = Porcentaje_empleo_desempleo_participacion_desempleados[(Porcentaje_empleo_desempleo_participacion_desempleados["codigo_pais"]=="OECD")].index
Porcentaje_empleo_desempleo_participacion_desempleados.drop(indices,inplace=True)

In [ ]:
indices = Porcentaje_empleo_desempleo_participacion_empleados[(Porcentaje_empleo_desempleo_participacion_empleados["pais"]=="OECD - Total")].index
Porcentaje_empleo_desempleo_participacion_empleados.drop(indices,inplace=True)

In [69]:
merge1 = pd.merge(Porcentaje_empleo_desempleo_participacion_empleados, Porcentaje_empleo_desempleo_participacion_desempleados, on=['codigo_pais','pais','anio','lugar_de_nacimiento'], how='outer')

In [70]:
merge2 = pd.merge(merge1, Porcentaje_empleo_desempleo_participacion_participacion, on=['codigo_pais','pais','anio','lugar_de_nacimiento'], how='outer')

In [71]:
merge2['lugar_de_nacimiento'].replace(to_replace='Foreign-born', value='extranjero', inplace=True)
merge2['lugar_de_nacimiento'].replace(to_replace='Native-born', value='nativo', inplace=True)


In [72]:
participacion= pd.read_csv(r'Etl\Tasa de participación en la fuerza de trabajo.csv')

In [73]:
participacion.drop(['source', 'source.label', 'indicator', 'indicator.label','sex', 'sex.label', 'classif1', 'classif2'], axis=1, inplace= True)

In [74]:
participacion.drop(columns=participacion.columns[6:15],axis=1,inplace= True)

In [75]:
participacion.rename(columns={'ref_area':'codigo_pais','ref_area.label':'pais',
                          'classif1.label':'edad', 'classif2.label':'lugar_de_nacimiento',
                          'time': 'anio', 'obs_value': 'tasa_participacion'}, inplace=True)

In [76]:
participacion= participacion[(participacion['edad']== 'Edad (Tramos agregados): Total')]

In [77]:
participacion.drop(['edad'], axis=1, inplace=True)

In [78]:
participacion['lugar_de_nacimiento'].replace(to_replace='Lugar de nacimiento: Nativo-nacido', value='nativo', inplace=True)
participacion['lugar_de_nacimiento'].replace(to_replace='Lugar de nacimiento: Nacido en el extranjero', value='extranjero', inplace=True)

In [81]:
fact_empleo_condicion = pd.merge(merge2, participacion, on=['codigo_pais','pais','lugar_de_nacimiento','anio'], how='outer')

In [82]:
fact_empleo_condicion

,codigo_pais,pais,lugar_de_nacimiento,anio,empleados,desempleados,participacion,tasa_participacion
0,AUS,Australia,extranjero,2000,64.4,6.7,69.0,NaN
1,AUS,Australia,extranjero,2001,64.0,7.3,69.0,NaN
2,AUS,Australia,extranjero,2002,64.5,6.8,69.2,NaN
3,AUS,Australia,extranjero,2003,64.9,6.4,69.3,NaN
4,AUS,Australia,extranjero,2004,66.0,5.5,69.9,NaN
...,...,...,...,...,...,...,...,...
3381,ZMB,Zambia,extranjero,2017,NaN,NaN,NaN,47.3
3382,ZWE,Zimbabwe,nativo,2019,NaN,NaN,NaN,65.9
3383,ZWE,Zimbabwe,extranjero,2019,NaN,NaN,NaN,53.6
3384,ZWE,Zimbabwe,nativo,2014,NaN,NaN,NaN,84.7
